In [1]:
# This Notebook Enriches the Joins the Officers and Teams tablue using the user teams mapping

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sys
sys.argv.append('--source_catalog_table')
# TODO: set name of the source table name in the glue catalog 
sys.argv.append('users')
# changed to full file

sys.argv.append('--source_catalog_table2')
# TODO: set name of the source table name in the glue catalog 
sys.argv.append('user_team_map')
# Changed to full file


sys.argv.append('--source_catalog_table3')
# TODO: set name of the source table name in the glue catalog 
sys.argv.append('user_teams')
# changed to full file


sys.argv.append('--source_catalog_database')
# TODO: set name of the source database name in the glue catalog 
sys.argv.append('dataplatform-stg-tascomi-refined-zone')

sys.argv.append('--s3_bucket_target')
# TODO: set S3 location where the transformed data will be saved
sys.argv.append('s3://dataplatform-stg-trusted-zone/tascomi/tables')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import sys
from awsglue.transforms import *
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from pyspark.sql.functions import *
import pyspark.sql.functions as F

# from helper.helpers import get_glue_env_var,

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default

glueContext = GlueContext(SparkContext.getOrCreate())
job = Job(glueContext)


# Define Source Data
source_catalog_table = get_glue_env_var('source_catalog_table', '')
source_catalog_table2 = get_glue_env_var('source_catalog_table2', '')
source_catalog_table3 = get_glue_env_var('source_catalog_table3', '')

source_catalog_database = get_glue_env_var('source_catalog_database', '')
target_bucket = get_glue_env_var('s3_bucket_target', '')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def drop_null_columns(df):
  
    _df_length = df.count()
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v >= _df_length]
    df = df.drop(*to_drop)
    
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_snapshot(df):
    
   df = df.where(col('snapshot_date') == df.select(max('snapshot_date')).first()[0])
   return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Load Officers Table
data_source = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table
 )

# convert to a data frame
df = data_source.toDF()


# Rename columns
df = df.withColumnRenamed("id","officer_id") \
       .withColumnRenamed("forename","officer_forename") \
       .withColumnRenamed("surname","officer_surname") 


# Specify Columns to Keep

df = df.select('officer_id',
               "officer_forename",
               "officer_surname",
               'username',
               'email',
               'mobile',
               'phone',
               'job_title',
               'import_date',
               'import_day',
               'import_month',
               'import_year',
               'snapshot_date')
              

# drop old snapshots

df = get_latest_snapshot(df)

# Drop Columns containing Only Nulls 
# df = drop_null_columns(df)

# Create Calculated Fields for Reporting

df = df.withColumn('counter_officer', lit(1))
df = df.withColumn('officer_name',concat(trim(col('officer_forename')),lit(" "),trim(col('officer_surname'))))

df.select('officer_name').show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|        officer_name|
+--------------------+
|       Sandy Ampomah|
|        Farhan Aleem|
|    Kehinde Ayorinde|
|    Mobolaji Sijuade|
| Jonathan Bainbridge|
|     Hesam Dorudvash|
|     Laurence Hughes|
|  Monika Vistartaite|
|           Mark Daly|
|Patrick O&#39;Connor|
|      Timothy Walder|
|      Peter Rathmell|
|     Benjamin Coffie|
|       Daryell Scott|
|   Marcin Manikowski|
|       Stuart Dunlop|
|    Tura Ajani-Salau|
|   Caroline Otubambo|
|   Rugiatu Parkinson|
|        Adele Castle|
+--------------------+
only showing top 20 rows

In [7]:
# Load User Teams Mapping Table
data_source = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table2
)

# convert to a data frame
df2 = data_source.toDF()

# drop old snapshots

df2 = get_latest_snapshot(df2)

# Rename Relevant Columns
# df2 = df2.withColumnRenamed("user_id","officer_id") 

# Keep Only Relevant Columns
df2 = df2.select("user_id",
                 "user_team_id")


# Check Result
# df2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Load Teams Table
data_source = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table3
)

# convert to a data frame
df3 = data_source.toDF()

# drop old snapshots

df3 = get_latest_snapshot(df3)

df3 = df3.withColumnRenamed("id","team_id") \
         .withColumnRenamed("name","team_name") \
         .withColumnRenamed("description","team_description")

# Keep Only Relevant Columns
df3 = df3.select("team_id","team_name",'team_description','location')

# Check Result
# df3.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Join

df2 = df2.join(df3,df2.user_team_id ==  df3.team_id,"left")
df = df.join(df2,df.officer_id ==  df2.user_id,"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df = df.drop("team_id","user_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+-------------------+--------------------+
|officer_id|team_id|       officer_name|           team_name|
+----------+-------+-------------------+--------------------+
|       496|      8|       Joshua Bwire|Technical Admin Team|
|       858|      8|   Tura Ajani-Salau|Technical Admin Team|
|      1025|   null|    Carmen Stephens|                null|
|       580|   null|  Marcin Manikowski|                null|
|      1005|     42|     Osaro Osagiede|Customer Service ...|
|      1016|   null|    Eleanor Winning|                null|
|       976|   null| Olatunde Feyisetan|                null|
|      1031|     42|     Nayyar Mehmood|Customer Service ...|
|       596|      4|       Martha Clark|        Enforcement |
|      1051|   null|          Maya Elmi|                null|
|       842|     40|     Timothy Walder|        Central Team|
|       842|     19|     Timothy Walder|          North Team|
|       842|     32|     Timothy Walder|Conservation, Urb...|
|      1

In [ ]:
# Transformation Ends

In [ ]:
# Convert data frame to dynamic frame 
dyanmic_frame = DynamicFrame.fromDF(df, glueContext, "target_data_to_write")

# Write the data to S3
parquet_data = glueContext.write_dynamic_frame.from_options(
    frame=dynamic_frame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_bucket, "partitionKeys": ["import_year", "import_month", "import_day", "import_date"]},
    transformation_ctx="target_data_to_write")